In [164]:
import random
import math
import copy

In [165]:
class Test(CRO):
    """
    This is a derived class to find the minimum value of sin(x) + cos(y).
    In real terms, you should implement this class according to your real problem.
    Note that every copy operation is deep-copy.
    """
    def __init__(self, structure):
        CRO.__init__(self, structure)

    def wall(self, m):
        new = copy.deepcopy(m)
        new.structure[0], new.structure[1] = new.structure[1], new.structure[0]
        return new

    def dec(self, m):
        new1 = copy.deepcopy(m)
        new2 = copy.deepcopy(m)
        new1.structure[0] += random.random()
        new2.structure[1] += random.random()
        return [new1, new2]

    def inter(self, m1, m2):
        new1 = copy.deepcopy(m1)
        new2 = copy.deepcopy(m2)
        new1.structure[0] = m2.structure[0]
        new1.structure[1] = m1.structure[1]
        new2.structure[0] = m1.structure[0]
        new2.structure[1] = m2.structure[1]
        return [new1, new2]

    def syn(self, m1, m2):
        new = copy.deepcopy(m1)
        if random.random() < 0.5:
            new.structure[0] = m2.structure[0]
        else:
            new.structure[1] = m2.structure[1]
        return new

    def fit_func(self, m):
            return math.sin(m.structure[0]) + math.cos(m.structure[1])

In [166]:
class Molecule:
    def __init__(self, structure):
        self.pe = 0
        self.ke = 0
        self.opt = 9999999
        self.num_of_hits = 0
        self.structure = copy.deepcopy(structure)

    def update(self):
        """
        If this molecule has a lower energy, reset num_of_hits.
        """
        if self.pe < self.opt:
            self.opt = self.pe
            self.num_of_hits = 0
    def __str__(self):
        reVal = ''
        for i in self.structure:
            reVal+=str(i)+" "
        return reVal


class CRO:
    """
    A base class represent the CRO algorithm.
    You should inherit from this class.
    """
    optimal = None  # A Molecule object represent the optimal solution
    cnt = 0
    limit = 1000
    KELossRate = 0.2
    mole_coll = 0.2
    alpha = 100
    beta = 100
    buffer = 0
    init_ke = 100
    pop = []

    def __init__(self, structure):
        """
        * fit_func: Object function
        * structure: Initial solution list [s1, s2, ..., sn]
        """
        for s in structure:
            self.pop.append(Molecule(s))

        for mol in self.pop:
            # You should implement this function in your derived class
            mol.pe = self.fit_func(mol)
            mol.ke = self.init_ke
            mol.update()
            if self.optimal is None:
                self.optimal = copy.deepcopy(mol)
            elif mol.pe < self.optimal.pe:
                self.optimal = copy.deepcopy(mol)

    def run(self):
        while self.cnt < self.limit:
            self.cnt += 1
            if random.random() > self.mole_coll or len(self.pop) < 2:
                m = random.choice(self.pop)
                if m.num_of_hits > self.alpha:
                    self.decomposition(m)
                else:
                    self.on_wall(m)
            else:
                m1, m2 = random.sample(self.pop, 2)
                if m1.ke <= self.beta and m2.ke <= self.beta:
                    self.synthesis(m1, m2)
                else:
                    self.interaction(m1, m2)

    def update(self, m):
        """
        If m is the current optimal solution, save it to the optimal.
        """
        if m.pe < self.optimal.pe:
            self.optimal = copy.deepcopy(m)

    def decomposition(self, m):
        m.num_of_hits += 1

        # You should implement this function in your derived class
        new1, new2 = self.dec(m)
        new1.pe = self.fit_func(new1)
        new2.pe = self.fit_func(new2)
        tmp = m.pe + m.ke - new1.pe - new2.pe
        if tmp >= 0 or tmp + self.buffer >= 0:
            if tmp >= 0:
                q = random.random()
                new1.ke = tmp * q
                new2.ke = tmp * (1 - q)
            else:
                new1.ke = (tmp + self.buffer) * random.random() * random.random()
                new2.ke = (tmp + self.buffer - new1.ke) * random.random() * random.random()
                self.buffer = self.buffer + tmp - new1.ke - new2.ke
            new1.update()
            new2.update()
            self.pop.remove(m)
            self.pop.append(new1)
            self.pop.append(new2)
            self.update(new1)
            self.update(new2)

    def on_wall(self, m):
        m.num_of_hits += 1

        # You should implement this function in your derived class
        new = self.wall(m)
        new.pe = self.fit_func(new)
        tmp = m.pe + m.ke - new.pe
        if tmp >= 0:
            q = random.uniform(self.KELossRate, 1)
            new.ke = tmp * q
            new.update()
            self.buffer += tmp * (1 - q)
            self.pop.remove(m)
            self.pop.append(new)
            self.update(new)

    def interaction(self, m1, m2):
        m1.num_of_hits += 1
        m2.num_of_hits += 1

        # You should implement this function in your derived class
        new1, new2 = self.inter(m1, m2)
        new1.pe = self.fit_func(new1)
        new2.pe = self.fit_func(new2)
        tmp = m1.pe + m2.pe + m1.ke + m2.ke - new1.pe - new2.pe
        if tmp >= 0:
            q = random.random()
            new1.ke = tmp * q
            new2.ke = tmp * (1 - q)
            new1.update()
            new2.update()
            self.pop.remove(m1)
            self.pop.remove(m2)
            self.pop.append(new1)
            self.pop.append(new2)
            self.update(new1)
            self.update(new2)

    def synthesis(self, m1, m2):
        m1.num_of_hits += 1
        m2.num_of_hits += 1

        # You should implement this function in your derived class
        new = self.syn(m1, m2)
        new.pe = self.fit_func(new)
        tmp = m1.pe + m2.pe + m1.ke + m2.ke - new.pe
        if tmp >= 0:
            new.ke = tmp
            new.update()
            self.pop.remove(m1)
            self.pop.remove(m2)
            self.pop.append(new)
            self.update(new)

In [167]:
s = [[random.random(), random.random()] for i in range(10)]
print(s)
cro = Test(s)
cro.run()
print(cro.optimal.pe)

[[0.3800599818155599, 0.3856326529570886], [0.38334828547671174, 0.6661160456099557], [0.16275922329625947, 0.5398813400951701], [0.40516704337762066, 0.2241413123441386], [0.6220000707077118, 0.662084613055528], [0.7875041055456901, 0.6158477427109108], [0.7132308253786828, 0.9433686143530275], [0.05526775630777481, 0.18857035401206534], [0.762722551669103, 0.6028469108711643], [0.5415937330219721, 0.9746974678765357]]
-1.9990084893550102


In [168]:
import random

# Define the function to generate supersequences
def generate_supersequences(input_strings, pop_size, superseq_length):
    population = []

    for _ in range(pop_size):
        superseq = ['-' for _ in range(superseq_length)]  # Start with an empty supersequence
        for string in input_strings:
            positions = sorted(random.sample(range(superseq_length), len(string)))  # Random unique positions
            for pos, char in zip(positions, string):
                superseq[pos] = char  # Place char in the supersequence
        
        # Convert the list of characters back to a string, replacing empty slots with random picks
        superseq = ''.join(char if char != '-' else random.choice('acgt') for char in superseq)
        population.append(superseq)
    
    return population

# Example usage:
input_strings = ['acg', 'cat', 'gtt']
pop_size = 20
superseq_length = 6  # Twice the length of the longest string

# Generate the population
initial_population = generate_supersequences(input_strings, pop_size, superseq_length)
initial_population  # Display the generated population


['acgttt',
 'ccgatt',
 'gaattt',
 'cgttgt',
 'agtatt',
 'gatcat',
 'agattg',
 'accgtt',
 'gatttc',
 'gtactg',
 'ggcttg',
 'gctagt',
 'cgtctt',
 'gtatgt',
 'gatctt',
 'gcatgt',
 'ccgatt',
 'cgagtt',
 'cgcttg',
 'cgattg']

# MY IMCRO

In [169]:
# Initialization


In [170]:
class MoleCule:
    def __init__(self, structure,supersequence):
        self.pe = 0
        self.ke = 0
        self.opt = 9999999
        self.num_of_hits = 0
        self.structure = copy.deepcopy(structure)
        self.supersequence = copy.deepcopy(supersequence)

    def update(self):
        """
        If this molecule has a lower energy, reset num_of_hits.
        """
        if self.pe < self.opt:
            self.opt = self.pe
            self.num_of_hits = 0
    def __str__(self):
        reVal = ''
        for i in self.structure:
            reVal+=str(i)+" "
        return reVal

In [171]:
def uni_moleReact():
    return
def inter_moleReact():
    return

In [172]:
# Iteration
i = 0
moleColl = 0.2
alpha = random.randint(10, 100)
beta = random.randint(10, 100)
while i!=1000:
    i+=1
    t = random.random()
    if t > moleColl:
        uni_moleReact()
    else :
        inter_moleReact()
    

In [173]:
import random

# Function to generate molecules with PE and KE
def generate_molecules(num_molecules, pe_range, ke_range, pe_ke_ratio):
    """
    Generates a list of molecules with random Potential Energy (PE) and Kinetic Energy (KE).
    The PE and KE values are generated with a constraint to keep them not too distinguishable.
    
    :param num_molecules: Number of molecules to generate.
    :param pe_range: Tuple (min, max) defining the range for PE.
    :param ke_range: Tuple (min, max) defining the range for KE.
    :param pe_ke_ratio: The ratio to ensure PE and KE are not too distinguishable.
    :return: List of dictionaries containing PE and KE for each molecule.
    """
    molecules = []
    for _ in range(num_molecules):
        # Generate PE and KE based on given ranges and ensure they are not too distinguishable using the ratio
        pe = random.uniform(*pe_range)
        # KE is constrained around PE value within a range modified by pe_ke_ratio
        ke = random.uniform(pe * pe_ke_ratio, pe * pe_ke_ratio + (ke_range[1] - ke_range[0]))
        molecules.append({'PE': pe, 'KE': ke})
    
    return molecules

# Example usage:
# Generate 20 molecules with PE between 10 and 20 and KE between 5 and 15, with a ratio of 0.8 to 1.2 for PE to KE
num_molecules = 20
pe_range = (10, 100)
ke_range = (10, 100)
pe_ke_ratio = 0.8  # Ratio to ensure PE and KE are closely matched

molecules = generate_molecules(num_molecules, pe_range, ke_range, pe_ke_ratio)
molecules  # Display the generated molecules with their PE and KE values


[{'PE': 18.930371405664072, 'KE': 16.45348365507815},
 {'PE': 47.63936167002003, 'KE': 80.4274644356997},
 {'PE': 73.58382521746412, 'KE': 147.6237946945414},
 {'PE': 12.450444111913198, 'KE': 27.441662743625898},
 {'PE': 89.44261581380938, 'KE': 140.13946359530473},
 {'PE': 51.839819425313436, 'KE': 47.397785843730176},
 {'PE': 41.62325879711605, 'KE': 49.54073197339258},
 {'PE': 43.64924300868166, 'KE': 79.14247393799675},
 {'PE': 19.585204752878376, 'KE': 27.550250689281377},
 {'PE': 68.89096081451783, 'KE': 80.49280175323742},
 {'PE': 86.3666939991997, 'KE': 101.25023804588662},
 {'PE': 20.49495036498955, 'KE': 61.01488763395906},
 {'PE': 43.33874840151071, 'KE': 85.72389497425627},
 {'PE': 16.344766607328932, 'KE': 63.24316735959366},
 {'PE': 51.71570993438765, 'KE': 45.38287269615226},
 {'PE': 95.26795715141014, 'KE': 105.04270000654934},
 {'PE': 79.56584564556519, 'KE': 127.261058894987},
 {'PE': 28.813655056414387, 'KE': 50.81827689758043},
 {'PE': 77.02881614253535, 'KE': 94.1

In [174]:
def find_scs(strings):
    def scs(X, Y):
        # Create a table to store lengths of longest common suffixes of substrings.
        # Note: A (m+1) x (n+1) table will be filled in bottom-up fashion.
        m, n = len(X), len(Y)
        L = [[0] * (n+1) for i in range(m+1)]

        # Fill the table in bottom-up manner
        for i in range(m+1):
            for j in range(n+1):
                if i == 0:
                    L[i][j] = j  # Min operations = j
                elif j == 0:
                    L[i][j] = i  # Min operations = i
                elif X[i-1] == Y[j-1]:
                    L[i][j] = 1 + L[i-1][j-1]
                else:
                    L[i][j] = 1 + min(L[i-1][j], L[i][j-1])

        # Following code is used to print the shortest common supersequence
        # scs will store the shortest supersequence
        index = L[m][n]
        scs = [""] * (index+1)
        scs[index] = ""

        # Start from the bottom right corner and one by one push characters in output string
        i, j = m, n
        while i > 0 and j > 0:
            # If current character in X and Y are same, then current character is part of supersequence
            if X[i-1] == Y[j-1]:
                scs[index-1] = X[i-1]
                i -= 1
                j -= 1
                index -= 1
            # If current character in X and Y are different, find the larger of two and go in the direction of larger value
            elif L[i-1][j] > L[i][j-1]:
                scs[index-1] = X[i-1]
                i -= 1
                index -= 1
            else:
                scs[index-1] = Y[j-1]
                j -= 1
                index -= 1

        # If Y reaches its end, put remaining characters of X in the result string
        while i > 0:
            scs[index-1] = X[i-1]
            i -= 1
            index -= 1

        # If X reaches its end, put remaining characters of Y in the result string
        while j > 0:
            scs[index-1] = Y[j-1]
            j -= 1
            index -= 1

        return "".join(scs)
    
    # Function to find SCS of a list of strings
    if not strings: return ""
    current_scs = strings[0]
    for string in strings[1:]:
        current_scs = scs(current_scs, string)
    return current_scs

# Example usage for the strings 'acg' and 'cat' and 'gtt':
scs_result = find_scs(['acg', 'cat', 'gtt'])
scs_result


'catagtcg'

In [175]:
# Redefine the function to generate an initial population for the IMCRO algorithm's SCS problem
def generate_population(set_of_strings, pop_size):
    population = []  # To store all generated supersequences (molecules)

    # Generate a population of supersequences
    for _ in range(pop_size):  # For each molecule in the population
        supersequence = ""  # Start with an empty supersequence for each molecule
        for string in set_of_strings:  # For each string in the set
            for character in string:  # For each character in the current string
                if character not in supersequence:  # Check if character is already included
                    supersequence += character  # Append character if not present
        population.append(supersequence)  # Add the generated supersequence to the population

    return population

# Given set of strings and population size for SCS problem
set_of_strings = ['acg', 'cat', 'gtt']
pop_size = 5

# Generate the initial population of supersequences
initial_population = generate_population(set_of_strings, pop_size)
initial_population


['acgt', 'acgt', 'acgt', 'acgt', 'acgt']

In [176]:
def insert_symbol(src_string,inserted_string,pos):
    return ''.join(src_string[:pos] + inserted_string + src_string[pos:])
# Given set of strings and population size for SCS problem
def supersequence_generate(set_of_strings):

    '''
        Make a copy of the set_of_strings parameter for maintaining the original 
        set
    '''
    copied_set_of_strings = copy.deepcopy(set_of_strings)
    supersequence = copied_set_of_strings.pop(random.randint(0,len(set_of_strings)-1))

    for i in range(len(copied_set_of_strings)):
        counter = 0
        for i in copied_set_of_strings[i]:
            inserted_pos = random.randint(counter,len(supersequence))
            if inserted_pos == len(supersequence):
                supersequence=insert_symbol(supersequence,i,inserted_pos)
                counter = inserted_pos+1
            elif i != supersequence[inserted_pos]:
                supersequence=insert_symbol(supersequence,i,inserted_pos)
                counter = inserted_pos+1

    return supersequence

# supersequence_generate(['acg', 'cat', 'gtt','tgc'])

def population_generation(pop_size,set_of_strings):
    l=[]
    for i in range(pop_size):
        l.append(supersequence_generate(set_of_strings))
    return l


# Generate many supersequence with popsize = 10 and given set of strings
population_generation(10,['acg', 'cat', 'gtt','tgc'])


['actcggatgttc',
 'caatgcggttc',
 'agcgtctagtc',
 'cgactgttgc',
 'gtgccacttagt',
 'cagtgcgactt',
 'aatgtcggct',
 'cagctgtgc',
 'atgctgatccg',
 'tgcccaggtt']